<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/Whisper_return_timestamp_bug_report_w_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-mnscvega
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-mnscvega
  Resolved https://github.com/huggingface/transformers.git to commit c836f77266be9ace47bff472f63caf71c0d11333
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Just for future reference, this test was we are on `transformers 4.27.0.dev0` at the time of testing.

In [3]:
!transformers-cli env

2023-02-13 08:10:14.258128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 08:10:17.547012: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-13 08:10:17.547203: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-13 08:10:17.547239: W tensorfl

In [9]:
from transformers import pipeline
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small", generate_kwargs={"task": "transcribe", "language": "<|de|>"})

Loading the CV11 german split in streaming mode

In [10]:
from datasets import load_dataset
cv11 = load_dataset("mozilla-foundation/common_voice_11_0", "de", streaming=True, split="test")

Checking the first element to see if the transcription actually is `german`

In [11]:
next(iter(cv11))

Reading metadata...: 16082it [00:00, 49621.91it/s]


{'client_id': '0052c07533a6976233ad5926d950b523002c4d8cdd9ae8726dbfec385951bd22aa707a742c49afe20c7d6cb9515dbaddac5b4d6fe8ebddcfbec46a2d3180a3a1',
 'path': 'common_voice_de_17922420.mp3',
 'audio': {'path': 'common_voice_de_17922420.mp3',
  'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         -9.0749630e-12,  5.6385865e-09,  7.3282314e-09], dtype=float32),
  'sampling_rate': 48000},
 'sentence': 'Zieht euch bitte draußen die Schuhe aus.',
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'de',
 'segment': ''}

Looks pretty german to me. In case you missed it, look at the `sentence` key.

Now, let's create a test sample from this sample.

In [12]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 57288.45it/s]
Reading metadata...: 16082it [00:00, 56202.61it/s]


In [13]:
pipe(test_sample, chunk_length_s=30)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1268: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' Zieht euch bitte draußen die Schuhe aus.'}

As seen above the pipeline simply refuses to accept the defined `generate_kwargs`

Let's try with `return_timestamps=True` and see how the response to this is.

In [14]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 22873.59it/s]
Reading metadata...: 16082it [00:00, 20316.21it/s]


In [15]:
pipe(test_sample, return_timestamps=True, chunk_length_s=30, stride_length_s=[6,0])

{'text': ' Zieht euch bitte draußen die Schuhe aus.',
 'chunks': [{'text': ' Zieht euch bitte draußen die Schuhe aus.',
   'timestamp': (0.0, 3.0)}]}

Now, the pipeline works, however, completely refuses to accept the `generate_kwargs` i.e. doesn't transcribe in german.

Attempt 2: Let's remove the `generate_kwargs` and try with `forced_decoder_ids`.

In [16]:
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

In [17]:
pipe.model.config.forced_decoder_ids = (
    pipe.tokenizer.get_decoder_prompt_ids(
        language="de", task="transcribe"
    )
)

In [18]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 45812.71it/s]
Reading metadata...: 16082it [00:00, 55874.44it/s]


In [19]:
pipe(test_sample, chunk_length_s=30)

{'text': ' Zieht euch bitte draußen die Schuhe aus.'}

Okay, that seem to work perfectly. Let's try and use the same with `return_timestamps=True`

In [20]:
test_sample = {"raw": next(iter(cv11))["audio"]["array"],
               "sampling_rate": next(iter(cv11))["audio"]["sampling_rate"]}

Reading metadata...: 16082it [00:00, 58640.02it/s]
Reading metadata...: 16082it [00:00, 49153.46it/s]


In [21]:
pipe(test_sample, return_timestamps=True, chunk_length_s=30, stride_length_s=[6,0])

{'text': ' Zieht euch bitte draußen die Schuhe aus.',
 'chunks': [{'text': ' Zieht euch bitte draußen die Schuhe aus.',
   'timestamp': (0.0, 3.0)}]}

Um, again, `translates` to english.